In [ ]:
#!pip install nisanyan-cli

In [ ]:
#!pip uninstall nisanyan-cli -y

In [ ]:
#!pip install scrapy

In [ ]:
#!pip install orjson

In [ ]:
#!pip install --upgrade pip

In [1]:
import scrapy
import pandas as pd
import numpy as np
import glob
import re
import nltk
from nltk import word_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from urllib.parse import quote
from urllib.request import Request
from urllib import parse
import orjson

In [8]:
# -*- coding: utf-8 -*-
# import argparse

#import requests
#from requests.adapters import HTTPAdapter
#from urllib3.util import Retry
#from urllib.parse import quote
#
#from bs4 import BeautifulSoup
#import orjson

# data dictionary
data_dict = {}

# Given words
first_word = "ab"
last_word = "zürriyet"

# Filenames
f_output = "output.json"
f_wordlist = "wordlist.txt"

# Request
def req(word):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    url = "https://www.nisanyansozluk.com/?k=" + quote(word) + "&lnk=1&view=annotated"
    session.mount(url, adapter)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
        "Accept-Encoding": "*",
        "Connection": "keep-alive",
        "accept-encoding": "gzip, deflate, br",
        "cache-control": "max-age=0",
        "content-type": "application/x-www-form-urlencoded",
        "dnt": "1",
        "upgrade-insecure-requests": "1",
    }
    r = session.get(url, headers=headers)
    # print(session.cookies.get_dict())
    return r.content


# Sometimes distortions occur when going sequentially in the table. e.g.: add -> "adem"
def back_forward():
    try:
        soup = BeautifulSoup(req(list(data_dict)[-2]), "html5lib")
        cell = soup.find("tr", {"class": "yaz hghlght"})
        nextt_word = cell.nextSibling.nextSibling.nextSibling.nextSibling.td["title"]

        print(
            "back_forward: "
            + list(data_dict)[-2]
            + " => \033[95m"
            + list(data_dict)[-1]
            + "\033[0m => "
            + nextt_word
        )
    except AttributeError:
        soup = BeautifulSoup(req(list(data_dict)[-3]), "html5lib")
        cell = soup.find("tr", {"class": "yaz hghlght"})
        nextt_word = cell.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.nextSibling.td[
            "title"
        ]
        print(
            "back_back_forward: "
            + list(data_dict)[-3]
            + " => \033[95m"
            + list(data_dict)[-2]
            + ", "
            + list(data_dict)[-1]
            + "\033[0m => "
            + nextt_word
        )
    gg(nextt_word)


# final message
def final_msg(msg):
    p_green = "\033[92m"
    p_blue = "\033[94m"
    p_endc = "\033[0m"

    print("-------------------------\n")
    print(p_blue + msg + p_endc + "\n")
    print(
        p_blue
        + "İlk kelime: "
        + p_green
        + list(data_dict)[0]
        + p_blue
        + ", Son kelime: "
        + p_green
        + list(data_dict)[-1]
    )
    print(p_blue + "Toplam kelime sayısı: " + p_green + str(len(data_dict)) + p_endc)
    print(p_blue + "Çıktı dosyası: " + p_green + f_output + p_endc)
    print(p_blue + "Kelime listesi dosyası: " + p_green + f_wordlist + p_endc)
    print("\a")


def export():
    # export data to .json file
    with open(f_output, "wb") as f:
        f.write(orjson.dumps(data_dict))

    # export wordlist to .txt file
    with open(f_wordlist, "w", encoding="utf-8") as f:
        f.write("\n".join(list(data_dict)))


# scrape the cell
def scrape_cell(cell):
    cell = cell
    kelime = cell.td["title"]
    tarih = cell.find("div", {"class": "maddetarih"}).text
    eskoken = cell.find_all("div", {"class": "eskoken"})
    baslik = (
        tarihce
    ) = koken = daha_fazla = ek_aciklama = benzer_sozcukler = maddeye_gonderenler = ""
    baslik = cell.td.a.text

    for i in eskoken:
        if i.div != None:
            title = i.div.text
            if "Tarihçe" in title:
                tarihce = str(i.p)
            elif "Köken" in title:
                koken = str(i.p)
            elif "Ek açıklama" in title:
                ek_aciklama = str(i.p)
            elif "Benzer sözcükler" in title:
                benzer_sozcukler = list(i.p.text.split(", "))
            elif "Bu maddeye gönderenler" in title:
                maddeye_gonderenler = list(i.p.text.strip().split(", "))
        elif "Daha fazla bilgi" in i.p.text:
            k = []
            a = i.p.find_all("a")
            for i in a:
                k.append(i.text)
            daha_fazla = k

    data = {
        "baslik": baslik,
        "tarihce": tarihce,
        "koken": koken,
        "daha_fazla": daha_fazla,
        "ek_aciklama": ek_aciklama,
        "benzer_sozcukler": benzer_sozcukler,
        "maddeye_gonderenler": maddeye_gonderenler,
        "tarih": tarih,
    }
    data_dict[kelime] = data
    print(kelime)


# Main function gg
def gg(first, last):
    word = first
    lastw = last
    print(word + " ➔ " + lastw + "\n")

    done = False

    while not done:
        content = req(word)
        soup = BeautifulSoup(content, "html.parser")
        cell = soup.find("tr", {"class": "yaz hghlght"})
        all_cells = soup.find_all("tr", {"class": "yaz hghlght"})

        # Wrong word
        if len(all_cells) == 0:
            print("Girilen kelime bulunamadı!")
            break
        # Double word e.g.: ram,RAM
        elif len(all_cells) > 1:
            for i in all_cells:
                scrape_cell(i)
        else:
            scrape_cell(cell)

        kelime = cell.td["title"]
        if kelime == lastw:
            done = True
        else:

            if (
                cell.nextSibling.nextSibling != None
                and cell.nextSibling.nextSibling.nextSibling.nextSibling != None
            ):
                if (
                    cell.nextSibling.nextSibling.find("div", {"class": "etymtxt"}).text
                    == ""
                ):
                    word = cell.nextSibling.nextSibling.nextSibling.nextSibling.td[
                        "title"
                    ]
                    continue
                word = cell.nextSibling.nextSibling.td["title"]
            else:
                back_forward()
    else:
        export()
        final_msg("Başarıyla tamamlandı...")
        exit()


def main(first=first_word, last=last_word):
    try:
        gg(first, last)
    except KeyboardInterrupt:
        export()
        final_msg("Liste tamamlanamadı!\nKeyboardInterrupt")
    except Exception as e:
        export()
        final_msg("Liste tamamlanamadı!\nBeklenmedik bir durum oluştu: " + str(e))

In [9]:
main()

ab ➔ zürriyet



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Girilen kelime bulunamadı!


In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
from urllib.parse import quote

from bs4 import BeautifulSoup
import orjson

In [ ]:
def req(word):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    url = "https://www.nisanyansozluk.com/?k=" + quote(word) + "&lnk=1&view=annotated"
    session.mount(url, adapter)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
        "Accept-Encoding": "*",
        "Connection": "keep-alive",
        "accept-encoding": "gzip, deflate, br",
        "cache-control": "max-age=0",
        "content-type": "application/x-www-form-urlencoded",
        "dnt": "1",
        "upgrade-insecure-requests": "1",
    }
    r = session.get(url, headers=headers)
    # print(session.cookies.get_dict())
    return r.content

In [ ]:
req("elma")

#### Arabic Language Process

In [ ]:
df_clean = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv")
#df_clean = df_clean.head(150000)
df_clean

In [ ]:
#df_test = df_clean[df_clean.lang == "Arabic"]
#df_test

In [ ]:
#df_test["term"].nunique()

In [ ]:
df_related_arabic = df_clean[df_clean.related_lang == "Arabic"]
df_related_arabic

In [ ]:
df_select_lang = df_related_arabic[df_related_arabic.lang == "Turkish"]
df_select_lang

In [ ]:
df_var1 = pd.DataFrame(df_select_lang["term"].unique(), columns=["term"])
df_var1

In [ ]:
df_var1.to_excel("Turkish_Term.xlsx", index=False)

In [ ]:
df_var2 = pd.DataFrame(df_select_lang["related_term"].unique(), columns=["related_term"])
df_var2

In [ ]:
df_var2.to_excel("Arabic_Related_Term.xlsx", index=False)

In [ ]:
df_var1_translate = pd.read_excel("Turkish Arabic Language Pair Translate.xlsx")
df_var1_translate

In [ ]:
df_var2.rename(columns={"related_term":"arabic_word"}, inplace=True)
df_var2

In [ ]:
pd.merge(df_var2,df_var1_translate, how="inner",on="arabic_word")

In [ ]:
df_var2_translate = pd.read_excel("Arabic Turkish Language Pair Translate.xlsx")
df_var2_translate

In [ ]:
df_var1.rename(columns={"term":"turkish_word"}, inplace=True)
df_var1

In [ ]:
pd.merge(df_var1,df_var2_translate, how="inner",on="turkish_word")

#### Etimoloji Türkçe

In [1]:
df_ety_sozluk = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/Turkish_0-40000_Word_Etimoloji_Result.xlsx")
df_ety_sozluk

<IPython.core.display.Javascript object>

,search_word,word,lang,koken,lang_root
0,bir,bir,Eski Türkçe,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,ve,Arapça,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" fi...",Arapça
2,için,i̇çin,Eski Türkçe,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğünd...",Eski Türkçe
3,çok,çok,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""bağ...",Ana Türkçe
4,ama,ama,Arapça,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağl...",Arapça
...,...,...,...,...,...
3701,masör,masör,Fransızca,"Fransızca masseur ""masaj yapan"" sözcüğünden al...",Fransızca
3702,baraka,baraka,İtalyanca,"İtalyanca baracca ""kulübe, derme çatma yapı"" s...",İtalyanca
3703,oğlak,oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tür...,Eski Türkçe
3704,boykot,boykot,İngilizce,"İngilizce boycott ""bir mal veya hizmeti satın ...",İngilizce


In [4]:
df_ety_sozluk[(df_ety_sozluk["lang"] == "Türkçe")]

,search_word,word,lang,koken,lang_root
19,sonra,sonra,Türkçe,Türkçe soŋ sözcüğünden +rA sonekiyle türetilmi...,Türkçe
31,başka,başka,Türkçe,Türkçe baş sözcüğünden +kA sonekiyle türetilmi...,Türkçe
37,hey,hey,Türkçe,Türkçe ünlemdir.,ünlemdir.
50,sorun,sorun,Türkçe,Türkçe sor- fiilinden +In sonekiyle türetilmiş...,Türkçe
61,haydi,haydi,Türkçe,Türkçe hay de! deyiminden evrilmiştir. Türkçe ...,Türkçe
...,...,...,...,...,...
3584,çiti,çiti,Türkçe,"Türkçe çit- ""elde çamaşır ovalamak"" fiilinden ...",Türkçe
3607,nere,nere,Türkçe,"Türkçe ne ""soru sıfatı ve zamiri"" sözcüğünden ...",Türkçe
3657,çekince,çekince,Türkçe,Türkçe çekin- fiilinden +çA sonekiyle türetilm...,Türkçe
3662,algı,algı,Türkçe,Türkçe al- fiilinden +gU sonekiyle türetilmişt...,Türkçe


In [6]:
df_ety_sozluk[(df_ety_sozluk["lang"] == "Türkçe") & (df_ety_sozluk["lang_root"] == "Türkçe")]

,search_word,word,lang,koken,lang_root
19,sonra,sonra,Türkçe,Türkçe soŋ sözcüğünden +rA sonekiyle türetilmi...,Türkçe
31,başka,başka,Türkçe,Türkçe baş sözcüğünden +kA sonekiyle türetilmi...,Türkçe
50,sorun,sorun,Türkçe,Türkçe sor- fiilinden +In sonekiyle türetilmiş...,Türkçe
61,haydi,haydi,Türkçe,Türkçe hay de! deyiminden evrilmiştir. Türkçe ...,Türkçe
64,kimse,kimse,Türkçe,Türkçe kim ise deyiminden türetilmiştir. Daha ...,Türkçe
...,...,...,...,...,...
3584,çiti,çiti,Türkçe,"Türkçe çit- ""elde çamaşır ovalamak"" fiilinden ...",Türkçe
3607,nere,nere,Türkçe,"Türkçe ne ""soru sıfatı ve zamiri"" sözcüğünden ...",Türkçe
3657,çekince,çekince,Türkçe,Türkçe çekin- fiilinden +çA sonekiyle türetilm...,Türkçe
3662,algı,algı,Türkçe,Türkçe al- fiilinden +gU sonekiyle türetilmişt...,Türkçe


#### Nişanyan

In [13]:
df_nisan = pd.read_excel("Nisanyan Words For English.xlsx")
df_nisan

,term,related_lang,related_term
0,+ark,Fransızca\n,+arque
1,+arşi,Fransızca\n,+archie
2,+faj,Fransızca\n ve İngilizce\n,+phage
3,+gam,Fransızca\n,+game
4,+gen,Fransızca\n,+gone
...,...,...,...
6012,zula,Yunanca,zúla
6013,zum,İngilizce\n,zoom
6014,zurna,eng,zurna
6015,اوطه‌لق,fra,odalisque


In [70]:
df_nisan["term"] = df_nisan["term"].apply(lambda x: x.strip("\+"))
df_nisan["related_term"] = df_nisan["related_term"].apply(lambda x: str(x).strip("\+"))
df_nisan

,term,related_lang,related_term
0,ark,Fransızca\n,arque
1,arşi,Fransızca\n,archie
2,faj,Fransızca\n ve İngilizce\n,phage
3,gam,Fransızca\n,game
4,gen,Fransızca\n,gone
...,...,...,...
6012,zula,Yunanca,zúla
6013,zum,İngilizce\n,zoom
6014,zurna,eng,zurna
6015,اوطه‌لق,fra,odalisque


In [5]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = df_word.head(10000)
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
164296,buldugumuzda,43
164297,boşandıklarını,42
164298,endişenlenme,42
164299,uzaylıymışım,42


In [6]:
set_nisanyan = set(df_nisan.term)
set_word = set(df_word.word)

In [10]:
df_nisan_word_intersect = pd.DataFrame(set_nisanyan.intersection(set_word), columns=["term"])
df_nisan_word_intersect

,term
0,roof
1,keton
2,kontrat
3,vantuz
4,gondol
...,...
2746,aort
2747,kukuleta
2748,lejyon
2749,hardcore


In [12]:
df_nisan_word_merge = pd.merge(df_nisan,df_nisan_word_intersect,how="inner",on="term")
df_nisan_word_merge

,term,related_lang,related_term
0,ark,Fransızca\n,ark
1,gam,Fransızca\n,gam
2,gen,Fransızca\n,gen
3,gen,fra,gen
4,gon,Fransızca\n,gon
...,...,...,...
2752,zooloji,Fransızca\n,zooloji
2753,zoraki,Yunanca,zoraki
2754,zula,Yunanca,zula
2755,zum,İngilizce\n,zum


### Turkish English Works

In [46]:
lang_1 = "Turkish"  # English, Turkish, German, French, Spanish, Arabic, Russian, Portuguese, Dutch, Italian 
lang_2 = "English"  # English, Turkish, German, French, Spanish, Arabic, Russian, Portuguese, Dutch, Italian

if lang_2 == "English":
    lang_3 = "English (eng)"
elif lang_2 == "Turkish":
    lang_3 = "Turkish (tur)"
elif lang_2 == "German":
    lang_3 = "German (deu)"
elif lang_2 == "French":
    lang_3 = "French (fra)"
elif lang_2 == "Spanish":
    lang_3 = "Spanish (spa)"
elif lang_2 == "Arabic":
    lang_3 = "Arabic (ara)"
elif lang_2 == "Russian":
    lang_3 = "Russian (rus)"
elif lang_2 == "Portuguese":
    lang_3 = "Portuguese (por)"
elif lang_2 == "Dutch":
    lang_3 = "Dutch (nld)"
elif lang_2 == "Italian":
    lang_3 = "Italian (ita)"
else:
    pass

print(f"{lang_1}\n{lang_2}\n{lang_3}")

Turkish
English
English (eng)


In [57]:
df_all_cross = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_1.capitalize()}/{lang_1.capitalize()}_All_Word_From_{lang_2.capitalize()}_Etymology.xlsx")
df_all_cross = df_all_cross.loc[:,["term_master","lang_sleeve","term_sleeve"]]
df_all_cross

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,reis,English,reis
2,espri,English,esprit
3,oksijen,English,oxygen
4,kaza,English,caza
...,...,...,...
93801,regülasyon,English,regulation
93802,levitasyon,English,gravitate
93803,haber spikeri,English,speaker
93804,radyo spikeri,English,radio


In [58]:
df_all_cross["term_master"].nunique()

2289

In [59]:
df_ety_turk = pd.read_excel("Etimolojiturk.xlsx")
df_ety_turk.rename(columns={"word":"term_master","lang_root":"lang_sleeve","koken":"term_sleeve"}, inplace=True)
df_ety_turk = df_ety_turk.loc[:,["term_master","lang_sleeve","term_sleeve"]]
df_ety_turk

,term_master,lang_sleeve,term_sleeve
0,kontrol,Latince,"Fransızca contrôle ""denetim"" sözcüğünden alınt..."
1,doktor,Latince,"Fransızca docteur ""1. üniversitede hocalık ber..."
2,polis,Eski Yunanca,"Fransızca police ""1. kamu düzeni [esk.], 2. po..."
3,dolar,Almanca,İngilizce aynı anlama gelen dollar sözcüğünden...
4,alo,Eski Fransızca,"Fransızca allo ""telefon hitabı"" sözcüğünden al..."
...,...,...,...
1180,şike,Fransızca,"Fransızca chiqué ""dalavereli"" sözcüğünden alın..."
1181,eşofman,Fransızca,"Fransızca échauffement ""ısınma, ısıtma, sporda..."
1182,masör,Fransızca,"Fransızca masseur ""masaj yapan"" sözcüğünden al..."
1183,baraka,İtalyanca,"İtalyanca baracca ""kulübe, derme çatma yapı"" s..."


In [60]:
df_dont_plus = pd.read_excel("Nisanyan Words Dont Contain Plus.xlsx")
df_dont_plus.rename(columns={"term":"term_master","related_lang":"lang_sleeve","related_term":"term_sleeve"}, inplace=True)
df_dont_plus

,term_master,lang_sleeve,term_sleeve
0,abajur,fra,abat-jour
1,abaküs,fra,abacus
2,abandone,Fransızca\n,s'abandonner
3,abdest,eng,abdest
4,abdomen,Latince,"abdomen, abdomin-"
...,...,...,...
5857,zula,Yunanca,zúla
5858,zum,İngilizce\n,zoom
5859,zurna,eng,zurna
5860,اوطه‌لق,fra,odalisque


In [61]:
df_github_ety_nisan_concat = pd.concat([df_all_cross,df_ety_turk,df_dont_plus], axis=0)
df_github_ety_nisan_concat.drop_duplicates(inplace=True)
df_github_ety_nisan_concat

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,reis,English,reis
2,espri,English,esprit
3,oksijen,English,oxygen
4,kaza,English,caza
...,...,...,...
5857,zula,Yunanca,zúla
5858,zum,İngilizce\n,zoom
5859,zurna,eng,zurna
5860,اوطه‌لق,fra,odalisque


In [62]:
df_github_ety_nisan_concat.to_excel("Turkish_English_Github_Etimoloji_Nisanyan_Concat_Result.xlsx", index=False)

In [63]:
set_df_cross = set(df_all_cross["term_master"])
set_df_ety_turk = set(df_ety_turk["term_master"])
set_df_nisan = set(df_dont_plus["term_master"])

In [64]:
df_turkish_english = pd.DataFrame((set_df_cross.union(set_df_ety_turk)).union(set_df_nisan), columns=["word_unique"])
df_turkish_english["word_unique"] = df_turkish_english["word_unique"].apply(lambda x: x.lower())
df_turkish_english

,word_unique
0,solvent
1,dvd
2,londralı
3,koli2
4,aferist
...,...
7166,monokl
7167,zero
7168,tatlı
7169,neon


In [54]:
df_turkish_english.to_excel("Turkish_English_All_Unique_Word.xlsx", index=False)

##### All Unique Word Github Intersect

In [24]:
df_github_cross = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv")
df_github_cross

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,inherited_from,9w90qvZDUH-2OHPVZEGJSQ,Middle english,parlai
1,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,Uy6v5wWuW6KGH4aTqGWnmA,Old french,parler
2,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,HY24XyCcUZCYSa1Q6y2kPA,Late latin,parabolō
3,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,uXGtk5g-W8G_TlTxSHImpg,Latin,parabola
4,LHuH1RvcUjWNUTR6QQF8fg,Spanish,palabra,derived_from,JHVmHqAPXEq-YaIbd0B5Jg,Ancient greek,παραβολή
...,...,...,...,...,...,...,...
8854119,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,inherited_from,TUnsO83vXXC1Us5rJgGyUg,Vulgar latin,*pittitus
8854120,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,oUnwLLsHVymmwwiRXgrRow,Latin,pitinnus
8854121,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,IGRZaHQ-VU-vpxMjdxSTlA,Latin,pitulus
8854122,3q28WGgPWs2K_SSglCAWvQ,Catalan,petitesa,etymologically_related_to,MDRrWciZWXGEj1KR9MUWtg,Latin,piccinus


In [25]:
df_github_turkish = df_github_cross[df_github_cross["lang"] == "Turkish"]
df_github_turkish

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
110,caAhC_cAXrKZU9rCcPAaPw,Turkish,göl,inherited_from,T6Wxr8YPVWK47B34DsG6KA,Proto-turkic,*kȫl
817,SC0fKM3-VcGTWvelEkpEXw,Turkish,ön,inherited_from,nlVlgr5dUjas0_4GqplLJQ,Proto-turkic,*ǖn
818,SC0fKM3-VcGTWvelEkpEXw,Turkish,ön,inherited_from,5g1DdR5OUPOxiQxCVcGW6A,Proto-turkic,*öŋ
819,SC0fKM3-VcGTWvelEkpEXw,Turkish,ön,inherited_from,6KC332NQU7Sx9sFKEv6QdQ,Proto-turkic,*oŋ
1345,5bVUD53NVEWYY0QAetV7iA,Turkish,net,inherited_from,Ru1rzGVDWpi2rTYD9-XJWQ,Old french,net
...,...,...,...,...,...,...,...
8853155,nxRVOzFvUzS4X7r165GWcw,Turkish,ordinatör,etymologically_related_to,tPEd3hNNXqeBt17YkY3I2Q,French,calculateur
8853651,pl-cU0pBVda2_BuEjBmtZw,Turkish,virman,has_prefix_with_root,9CJvE6k0WJmKvcWZQmQjxw,French,virer
8853652,pl-cU0pBVda2_BuEjBmtZw,Turkish,virman,has_suffix,8YtrMi5VXlSpZVOupUTcWg,French,ment
8853831,fc8biERQXFicQGtfESBJQQ,Turkish,angajman,has_prefix_with_root,qQY-6sksXJe832mC9wIBqQ,French,engager


In [26]:
df_turkish_english["word_unique"] = df_turkish_english["word_unique"].apply(lambda x: x.lower()) 

In [27]:
df_turkish_english = pd.read_excel("Turkish_English_All_Unique_Word.xlsx")
df_turkish_english

,word_unique
0,solvent
1,dvd
2,londralı
3,koli2
4,aferist
...,...
7166,monokl
7167,zero
7168,tatlı
7169,neon


In [28]:
set_github_turkish = set(df_github_turkish["term"])
set_all_unique = set(df_turkish_english["word_unique"])

In [29]:
df_all_unique_word_github = pd.DataFrame(set_github_turkish.intersection(set_all_unique), columns=["word_intersect"])
df_all_unique_word_github

,word_intersect
0,altın sinek
1,terim
2,kontrat
3,gondol
4,sör
...,...
2319,plaj
2320,tabure
2321,peyzaj
2322,tatlı


##### All Unique Word All Data Intersect

In [65]:
df_github_ety_nisan_concat = pd.read_excel("Turkish_English_Github_Etimoloji_Nisanyan_Concat_Result.xlsx")
df_github_ety_nisan_concat

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,reis,English,reis
2,espri,English,esprit
3,oksijen,English,oxygen
4,kaza,English,caza
...,...,...,...
100834,zula,Yunanca,zúla
100835,zum,İngilizce\n,zoom
100836,zurna,eng,zurna
100837,اوطه‌لق,fra,odalisque


In [66]:
df_turkish_english = pd.read_excel("Turkish_English_All_Unique_Word.xlsx")
df_turkish_english.rename(columns={"word_unique":"term_master"}, inplace=True)
df_turkish_english

,term_master
0,solvent
1,dvd
2,londralı
3,koli2
4,aferist
...,...
7166,monokl
7167,zero
7168,tatlı
7169,neon


In [67]:
df_merge_all = pd.merge(df_github_ety_nisan_concat,df_turkish_english, how="inner", on="term_master")
df_merge_all.drop_duplicates(inplace=True)
df_merge_all

,term_master,lang_sleeve,term_sleeve
0,sirk,English,cirque
1,sirk,English,crucial
2,sirk,English,cruciform
3,sirk,English,cross
4,sirk,English,incarcerate
...,...,...,...
101086,zoraki,Yunanca,zoráki
101087,zula,Yunanca,zúla
101088,zum,İngilizce\n,zoom
101089,اوطه‌لق,fra,odalisque


In [43]:
df_merge_all.to_excel("Turkish_English_All_Unique_Word_Related_Lang_And_Term.xlsx", index=False)

In [41]:
df_merge_all["term_master"].nunique()

7134

#### Nişanyan Words Contain Plus

In [3]:
import pandas as pd
import numpy as np
import glob
import re

In [4]:
df_word_plus_nisanyan = pd.read_excel("Nisanyan Words Contain Plus.xlsx")
df_word_plus_nisanyan

,term,related_lang,related_term
0,+ark,Fransızca\n,+arque
1,+arşi,Fransızca\n,+archie
2,+faj,Fransızca\n ve İngilizce\n,+phage
3,+gam,Fransızca\n,+game
4,+gen,Fransızca\n,+gone
...,...,...,...
150,tri+,Fransızca\n ve İngilizce\n,tri+
151,trop(o)+,Fransızca\n ve İngilizce\n,tropo+
152,üni+,Fransızca\n ve İngilizce\n,uni+
153,vaz(o)+,Fransızca\n ve İngilizce\n,vaso+


In [5]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = df_word.head(10000)
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
164296,buldugumuzda,43
164297,boşandıklarını,42
164298,endişenlenme,42
164299,uzaylıymışım,42


In [6]:
word = "+ark"

In [7]:
word.startswith("+")

True

In [8]:
word_list = df_word_plus_nisanyan.loc[:,"term"].values.tolist()

In [38]:
df_word_result = pd.DataFrame()
for i in word_list:
    if i.startswith("+"):
        j = i.lstrip("+")
        word_in_video = df_word[df_word.word.str.contains(fr"{j}(?:$)", na=True)]
        word_in_video.insert(0,"word_ext",i)
    elif i.endswith("+"):
        j = i.rstrip("+")
        word_in_video = df_word[df_word.word.str.contains(fr"(?:^){j}", na=True)]
        word_in_video.insert(0,"word_ext",i)
    else:
        pass
    #word_in_video.insert(0,"word_ext",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result = df_word_result.sort_values(by="frequency", ascending=False)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result        

<ipython-input-38-67b24e9e674c>:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  word_in_video = df_word[df_word.word.str.contains(fr"(?:^){j}", na=True)]


,word_ext,word,frequency
0,ö+,öyle,1440030
1,ö+,önce,1291540
2,+men,hemen,607973
3,di+,diye,562704
4,ö+,önemli,446014
...,...,...,...
6527,ö+,ölmeyince,53
6528,üni+,üniversitedeydin,52
6529,di+,disini,48
6530,dis+,disini,48


In [35]:
df_word_result.sample(20)

,word_ext,word,frequency
2318,ö+,öğrendiğiniz,532
4438,di+,dişlerden,187
4889,ö+,ölürlerdi,154
5791,dis+,disiplinine,98
1868,ö+,örtünün,734
137,ö+,öp,16042
3690,ö+,örnekleriyle,260
4277,di+,dirsekleri,201
4339,ö+,öğrendiklerin,195
2863,di+,diktatörün,385
